In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers
from keras.utils import to_categorical
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
# Any results you write to the current directory are saved as output.

Viewing one image

In [ ]:
test = image.load_img('/kaggle/input/train data/Train data/leukocoria/0.jpg', target_size=(30,30,3))
plt.imshow(test)
plt.show()

Finding the location of files

In [ ]:
import os
os.getcwd()
os.listdir('/kaggle')
#train = pd.read_csv('../input/Train data.rar')
os.listdir('/kaggle/input/train data/Train data')


In [ ]:
Files =  os.listdir('/kaggle/input/train data/Train data/leukocoria')
train_image = []
for i in Files:
    img = image.load_img('/kaggle/input/train data/Train data/leukocoria/'+i, target_size=(30,30,3), color_mode='grayscale')
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

The size of the training data set


In [ ]:
np.shape(X)

**Class 1 Y**

In [ ]:
y_train_leukocoria = np.ones(np.shape(X)[0])
np.shape(y_train_leukocoria)

**loading the 0 class**

In [ ]:
Files_0 =  os.listdir('/kaggle/input/train data/Train data/non-leukocoria')
train_image_0 = []
for i in Files_0:
    img = image.load_img('/kaggle/input/train data/Train data/non-leukocoria/'+i, target_size=(30,30,3), 
                         color_mode='grayscale')
    img = image.img_to_array(img)
    img = img/255
    train_image_0.append(img)
X_0 = np.array(train_image_0)

**Class 0 Y**

In [ ]:
y_train_non_leukocoria = np.zeros(np.shape(X_0)[0])
np.shape(y_train_non_leukocoria)

In [ ]:
train_x = np.concatenate((X,X_0), axis=0)
train_y = np.concatenate((y_train_leukocoria,y_train_non_leukocoria), axis=0)

In [ ]:
print(np.shape(train_x))
print(np.shape(train_y))

**test train split**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, random_state=42, test_size=0.2)

**Check the Y data split**

In [ ]:
print('Before Stratification Y_train \n',pd.crosstab(y_train,columns='count'))

**Create categorical data using keras to_categorical**

In [ ]:
train_y_binary = to_categorical(train_y)

In [ ]:
type(train_y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, random_state=42, test_size=0.2, stratify = train_y )

In [ ]:
print('After Stratification Y_train \n',pd.crosstab(y_train,columns='count'))

**Model**

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(30,30,1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

**Compile the model**

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

**run model**

In [ ]:
np.shape(X_train)
np.shape(y_train)

In [ ]:
history = model.fit(X_train, y_train, epochs=2,validation_data=(X_test, y_test))

**Plot  Training and Validation accuracy**

In [ ]:
history_dict = history.history
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']

epochs = range(1, len(history_dict['acc']) +1)
plt.plot(epochs, acc_values, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc_values, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()

plt.show()

In [ ]:
#os.listdir('/kaggle/input/evaluation data/Evaluation data')

In [ ]:
import zipfile
#zip_ref = zipfile.ZipFile('/kaggle/input/evaluation data/Evaluation data.rar', 'r')
#zip_ref.extractall('/kaggle/input/evaluation data/Evaluation data')
#zip_ref.close()

In [ ]:
Files_test =  os.listdir('/kaggle/input/evaluation data/Evaluation data')
test_image = []
for i in Files_test:
    img = image.load_img('/kaggle/input/evaluation data/Evaluation data/'+i, target_size=(30,30,3), 
                         color_mode='grayscale')
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
X_2 = np.array(test_image)

In [ ]:
model.fit(X_train, y_train, epochs=10,validation_data=(X_test, y_test))
#model.fit(np.concatenate((X_train,X_test),axis=0), np.concatenate((y_train,y_test),axis=0), epochs=12,batch_size=50,validation_data=(X_test, y_test))

In [ ]:
import re
classes_out = model.predict_classes(X_2)
id = [re.sub('.jpg','',x) for x in Files_test]

submission = pd.DataFrame(
    {'Id' : id,
     'Category' : list(classes_out.reshape(200,))})


In [ ]:
submission.head()
submission.to_csv('Submission.csv', index=False)

**Augumented data Set**

In [ ]:
batch_size = 32

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2)

# this is the augmentation configuration we will use for testing:
# only rescaling
train_datagen.fit(train_x)

validation_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2)

validation_datagen.fit(train_x)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
model.fit_generator(train_datagen.flow(train_x, train_y, batch_size=32),
                    validation_data=validation_datagen.flow(train_x, train_y, batch_size=32),
                    steps_per_epoch=len(train_x)/32, epochs=20,
                     validation_steps=len(train_x)/32)

